In [1]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [2]:
import json
from src.surquest.utils.appstoreconnect.credentials import Credentials
from src.surquest.utils.appstoreconnect.analyticsreports.client import Client
from src.surquest.utils.appstoreconnect.analyticsreports.handler import Handler
from src.surquest.utils.appstoreconnect.analyticsreports.enums.category import Category
from src.surquest.utils.appstoreconnect.analyticsreports.enums.granularity import Granularity
from pathlib import Path

In [4]:
key_path = Path.cwd() / "credentials" / "key.p8"
CATEGORY = Category.APP_USAGE
REPORT_NAME = "App Store Installation and Deletion Detailed"
GRANULARITY = Granularity.DAILY
DATE = "2025-07-27"
APP_ID = "950949627"

KEY_ID = '5WDUV3USAU' # 10-character key ID from App Store Connect
ISSUER_ID = '69a6de80-fd44-47e3-e053-5b8c7c11a4d1'  # 36-character issuer ID
PRIVATE_KEY = key_path.read_text()


credentials = Credentials(
    issuer_id=ISSUER_ID, # 36-character issuer ID
    key_id=KEY_ID, # 10-character key ID from App Store Connect
    private_key=PRIVATE_KEY
)
client = Client(credentials=credentials)
# token = credentials.generate_token()
#1A Read Report Requests
# report_requests = client.read_report_requests(app_id=APP_ID)
# data = client.get_data(
#     app_id = APP_ID,
#     report_name = REPORT_NAME,
#     category = CATEGORY,
#     granularity = GRANULARITY,
#     # dates = {'2025-07-25', '2025-07-26'}
# )

data = client.list_report_types(
    app_id = APP_ID
)

print(json.dumps(Handler.extract_attribute_values(data), indent=4))



INFO     - 2025-08-03 10:12:18,720 - Initialized Client with provided credentials
INFO     - 2025-08-03 10:12:18,727 - Starting list_report_types process
DEBUG    - 2025-08-03 10:12:18,731 - Parameters: app_id=950949627
INFO     - 2025-08-03 10:12:18,734 - Fetching report requests for app_id: 950949627
DEBUG    - 2025-08-03 10:12:18,736 - Generated authorization token
DEBUG    - 2025-08-03 10:12:18,739 - GET https://api.appstoreconnect.apple.com/v1/apps/950949627/analyticsReportRequests | Params: None
DEBUG    - 2025-08-03 10:12:19,287 - Response Status: 200
DEBUG    - 2025-08-03 10:12:19,295 - Found report request IDs: ['2093eaa4-98e2-44ce-9e79-b796e45a3dd0']
INFO     - 2025-08-03 10:12:19,299 - Fetching reports for report request id: 2093eaa4-98e2-44ce-9e79-b796e45a3dd0
DEBUG    - 2025-08-03 10:12:19,304 - Generated authorization token
DEBUG    - 2025-08-03 10:12:19,309 - GET https://api.appstoreconnect.apple.com/v1/analyticsReportRequests/2093eaa4-98e2-44ce-9e79-b796e45a3dd0/reports

{'data': [{'type': 'analyticsReports', 'id': 'r39-2093eaa4-98e2-44ce-9e79-b796e45a3dd0', 'attributes': {'name': 'AirPlay Discovery Sessions', 'category': 'FRAMEWORK_USAGE'}, 'relationships': {'instances': {'links': {'self': 'https://api.appstoreconnect.apple.com/v1/analyticsReports/r39-2093eaa4-98e2-44ce-9e79-b796e45a3dd0/relationships/instances', 'related': 'https://api.appstoreconnect.apple.com/v1/analyticsReports/r39-2093eaa4-98e2-44ce-9e79-b796e45a3dd0/instances'}}}, 'links': {'self': 'https://api.appstoreconnect.apple.com/v1/analyticsReports/r39-2093eaa4-98e2-44ce-9e79-b796e45a3dd0'}}, {'type': 'analyticsReports', 'id': 'r154-2093eaa4-98e2-44ce-9e79-b796e45a3dd0', 'attributes': {'name': 'Home Screen Widget Rotations', 'category': 'FRAMEWORK_USAGE'}, 'relationships': {'instances': {'links': {'self': 'https://api.appstoreconnect.apple.com/v1/analyticsReports/r154-2093eaa4-98e2-44ce-9e79-b796e45a3dd0/relationships/instances', 'related': 'https://api.appstoreconnect.apple.com/v1/analy

In [4]:
#1A Read Report Requests
report_requests = client.read_report_requests(app_id=APP_ID)
report_request_id = report_requests.get("data")[0].get("id")
RenderJSON(report_requests)


INFO     - 2025-08-03 08:56:44,874 - Fetching report requests for app_id: 950949627
DEBUG    - 2025-08-03 08:56:44,879 - Generated authorization token
DEBUG    - 2025-08-03 08:56:44,883 - GET https://api.appstoreconnect.apple.com/v1/apps/950949627/analyticsReportRequests | Params: None
DEBUG    - 2025-08-03 08:56:45,130 - Response Status: 200


In [5]:
#2A Read Report
report = client.read_report_for_specific_request(
    request_id=report_request_id,
    params={
        "filter[category]": CATEGORY.value,
        "filter[name]": REPORT_NAME
    }
)

report_id = report.get("data")[0].get("id")
print(report_id)
print(report)
RenderJSON(report)

INFO     - 2025-08-03 08:56:45,146 - Fetching reports for report request id: 2093eaa4-98e2-44ce-9e79-b796e45a3dd0
DEBUG    - 2025-08-03 08:56:45,149 - Generated authorization token
DEBUG    - 2025-08-03 08:56:45,150 - GET https://api.appstoreconnect.apple.com/v1/analyticsReportRequests/2093eaa4-98e2-44ce-9e79-b796e45a3dd0/reports | Params: {'filter[category]': 'APP_USAGE', 'filter[name]': 'App Store Installation and Deletion Detailed'}
DEBUG    - 2025-08-03 08:56:45,347 - Response Status: 200


r7-2093eaa4-98e2-44ce-9e79-b796e45a3dd0
{'data': [{'type': 'analyticsReports', 'id': 'r7-2093eaa4-98e2-44ce-9e79-b796e45a3dd0', 'attributes': {'name': 'App Store Installation and Deletion Detailed', 'category': 'APP_USAGE'}, 'relationships': {'instances': {'links': {'self': 'https://api.appstoreconnect.apple.com/v1/analyticsReports/r7-2093eaa4-98e2-44ce-9e79-b796e45a3dd0/relationships/instances', 'related': 'https://api.appstoreconnect.apple.com/v1/analyticsReports/r7-2093eaa4-98e2-44ce-9e79-b796e45a3dd0/instances'}}}, 'links': {'self': 'https://api.appstoreconnect.apple.com/v1/analyticsReports/r7-2093eaa4-98e2-44ce-9e79-b796e45a3dd0'}}], 'links': {'self': 'https://api.appstoreconnect.apple.com/v1/analyticsReportRequests/2093eaa4-98e2-44ce-9e79-b796e45a3dd0/reports?filter%5Bname%5D=App+Store+Installation+and+Deletion+Detailed&filter%5Bcategory%5D=APP_USAGE'}, 'meta': {'paging': {'total': 1, 'limit': 50}}}


In [6]:
#3A Read Report Instances
instances = client.read_list_of_instances_of_report(
    report_id=report_id,
    params={
        "filter[granularity]": GRANULARITY.value,
        "filter[processingDate]": DATE
    }
)
instance_id = instances.get("data")[0].get("id")
RenderJSON(instances)

INFO     - 2025-08-03 08:56:45,363 - Fetching instances for report id: r7-2093eaa4-98e2-44ce-9e79-b796e45a3dd0 with params: {'filter[granularity]': 'DAILY', 'filter[processingDate]': '2025-07-27'}
DEBUG    - 2025-08-03 08:56:45,365 - Generated authorization token
DEBUG    - 2025-08-03 08:56:45,367 - GET https://api.appstoreconnect.apple.com/v1/analyticsReports/r7-2093eaa4-98e2-44ce-9e79-b796e45a3dd0/instances | Params: {'filter[granularity]': 'DAILY', 'filter[processingDate]': '2025-07-27'}
DEBUG    - 2025-08-03 08:56:45,624 - Response Status: 200


In [7]:
#4 Get segments
segments = client.read_segments_for_report(
    instance_id=instance_id
)
url  = segments.get("data")[0].get("attributes").get("url")
RenderJSON(segments)


INFO     - 2025-08-03 08:56:45,640 - Fetching segments for instance id: e6e1b0d7-2044-4f49-a109-64b2e0edaf28
DEBUG    - 2025-08-03 08:56:45,643 - Generated authorization token
DEBUG    - 2025-08-03 08:56:45,644 - GET https://api.appstoreconnect.apple.com/v1/analyticsReportInstances/e6e1b0d7-2044-4f49-a109-64b2e0edaf28/segments | Params: None
DEBUG    - 2025-08-03 08:56:45,847 - Response Status: 200


In [8]:
# Download the report data
data = client.download_report_to_dicts(
    report_url=url
)
data

INFO     - 2025-08-03 08:56:45,863 - Downloading report from URL: https://asp-us-west-2.s3.us-west-2.amazonaws.com/reports/950949627/app_installs_deletes_detailed/daily/ongoing/20250727/1753594021981/72nc70s7sf7qo.csv.gz?response-content-disposition=attachment%3B%20filename%3D%22950949627_App%20Store%20Installation%20and%20Deletion%20Detailed_Daily_2025-07-27_5e100792-0b1d-480e-8340-5e39ec2c6729.csv.gz%22&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEPH%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLXdlc3QtMiJGMEQCICh57%2F34pBnQ6z10iaS20l7DTraHGToHib0BE7kz7NFeAiBsA9IjyGkqjh2P9eOhvF2XQKXbPqQMuqKDtIxW%2FBkcmSqiAggqEAAaDDU3OTg1MDY5ODg2OCIM9eNWei39d3AgoyIcKv8BPuT%2FtMw7D7YkZHdWjKiX39KePbjQqgBcXz5mXMZ5IPI%2BtDt2DpHWX04gjJgS9newPtf4thg8COZ3hMX9wfwIf8m5p2jfCuMkOFJcR%2B3JHe%2F7s8BIV8UeBuourpkKjqTzgtoicX5uSqOzQ22Y2dhv1J1jERJUTuPwQWJZpJHRmmHhZJ%2FlWPGR7Msk18HPz2o5nG%2F2ET3nveBoG8mS1cngpOjstUS7XyFi5nw%2Bv%2BSVBRbCU9TKGVbtkFAYUdKkwb%2BfQ5ONIqpidaB5xiu27kCrViNcFLcfmHU%2B%2F7ywW6Mdp1Ux%2B5%2BjXksNvieyI2BwjFVfPnKiUlc5

[{'date': '2025-07-22',
  'app_name': 'Landlord Tycoon: Own the World',
  'app_apple_identifier': '950949627',
  'event': 'Delete',
  'download_type': '',
  'app_version': '4.10.8',
  'device': 'iPhone',
  'platform_version': 'iOS 18.5',
  'source_type': 'App Store search',
  'source_info': '',
  'campaign': '',
  'page_type': 'No page',
  'page_title': 'Default No Page',
  'app_download_date': '',
  'territory': 'DE',
  'counts': '6',
  'unique_devices': '6'},
 {'date': '2025-07-22',
  'app_name': 'Landlord Tycoon: Own the World',
  'app_apple_identifier': '950949627',
  'event': 'Delete',
  'download_type': '',
  'app_version': '4.11.0',
  'device': 'iPhone',
  'platform_version': 'iOS 18.5',
  'source_type': 'App Store search',
  'source_info': '',
  'campaign': '',
  'page_type': 'No page',
  'page_title': 'Default No Page',
  'app_download_date': '2025-07-21',
  'territory': 'US',
  'counts': '7',
  'unique_devices': '7'},
 {'date': '2025-07-22',
  'app_name': 'Landlord Tycoon: Ow

In [9]:
client.get_data(
    app_id = APP_ID,
    report_name = REPORT_NAME,
    category = CATEGORY,
    granularity = GRANULARITY,
    dates = {'2025-07-25', '2025-07-26'}
)


INFO     - 2025-08-03 08:56:46,071 - Starting get_data process
DEBUG    - 2025-08-03 08:56:46,074 - Parameters: app_id=950949627, report_name=App Store Installation and Deletion Detailed, category=Category.APP_USAGE, granularity=Granularity.DAILY, dates={'2025-07-25', '2025-07-26'}
INFO     - 2025-08-03 08:56:46,075 - Fetching report requests for app_id: 950949627
DEBUG    - 2025-08-03 08:56:46,077 - Generated authorization token
DEBUG    - 2025-08-03 08:56:46,080 - GET https://api.appstoreconnect.apple.com/v1/apps/950949627/analyticsReportRequests | Params: None
DEBUG    - 2025-08-03 08:56:46,300 - Response Status: 200
DEBUG    - 2025-08-03 08:56:46,302 - Found report request IDs: ['2093eaa4-98e2-44ce-9e79-b796e45a3dd0']
INFO     - 2025-08-03 08:56:46,303 - Fetching reports for report request id: 2093eaa4-98e2-44ce-9e79-b796e45a3dd0
DEBUG    - 2025-08-03 08:56:46,305 - Generated authorization token
DEBUG    - 2025-08-03 08:56:46,305 - GET https://api.appstoreconnect.apple.com/v1/analy

[{'date': '2025-07-22',
  'app_name': 'Landlord Tycoon: Own the World',
  'app_apple_identifier': '950949627',
  'event': 'Delete',
  'download_type': '',
  'app_version': '4.10.8',
  'device': 'iPhone',
  'platform_version': 'iOS 18.5',
  'source_type': 'App Store search',
  'source_info': '',
  'campaign': '',
  'page_type': 'No page',
  'page_title': 'Default No Page',
  'app_download_date': '',
  'territory': 'DE',
  'counts': '6',
  'unique_devices': '6'},
 {'date': '2025-07-22',
  'app_name': 'Landlord Tycoon: Own the World',
  'app_apple_identifier': '950949627',
  'event': 'Delete',
  'download_type': '',
  'app_version': '4.11.0',
  'device': 'iPhone',
  'platform_version': 'iOS 18.5',
  'source_type': 'App Store search',
  'source_info': '',
  'campaign': '',
  'page_type': 'No page',
  'page_title': 'Default No Page',
  'app_download_date': '2025-07-21',
  'territory': 'US',
  'counts': '7',
  'unique_devices': '7'},
 {'date': '2025-07-22',
  'app_name': 'Landlord Tycoon: Ow